[Reference](https://medium.com/better-programming/how-to-create-a-reusable-web-scraper-5a5aa9af62d9)

In [7]:
import requests
import pickle
from random import randint
from urllib.parse import urlparse
def _random_ua():
    ua_list = ["user agent1, user agent2", "user agent3"]
    random_num = randint(0, len(ua_list))
    return ua_list[random_num]
def _headers():
    return { 'user-agent': _random_ua() }
def _save_page(response):
    uri = urlparse(response.url)
    filename = uri.netloc + ".pickle"
    with open(filename, 'wb+') as pickle_file:            
        pickle.dump(response, pickle_file)
def download_page(url):
    response = requests.get(url)
    _save_page(request)

In [10]:
def outer_element(page, identifyer):
    root = page.find(*identifyer)
    
    if root == None:
        raise Exception("Could not find root element")

    return root


# Remove unwanted elements
def trim_unwanted(page, identifier_list):
    # Check if list has elements
    if len(identifier_list) != 0:
        for identifier in identifier_list:
            for element in page.find_all(*identifier):
                element.decompose()
    return page


# Extract text
def get_text(page, identifier_list):
    # Check if list has elements
    if len(identifier_list) == 0:
        raise Exception("Need text elements")
    page_text = []
    
    for identifier in identifier_list:
        for element in page.find_all(*identifier):
            page_text.append(element.text)
        return page_text


# Get page config
def load_scrape_config():
    '''Loads page scraping config data'''
    return get_scrape_config()  


# Get the scraping config for the site
def get_site_config(url):
    '''Get the scrape config for the site'''
    domain = extract_domain(url)
    config_data = load_scrape_config()
    config = config_data.get(domain, None)
    if config == None:
        raise Exception(f"Config does not exist for: {domain}")
    return config 


# Build Soup
def page_processer(request):
    '''Returns Article Text'''
    # Get the page scrape config
    site_config = get_site_config(request.url)     
    
    # Soupify page
    soup = BeautifulSoup(request.text, 'lxml')
    
    # Retrieve root element    
    root = outer_element(soup, site_config["root_element"])     
    # Remove unwanted elements
    trimmed_tree = trim_unwanted(root, site_config["unwanted"])
    # Get the desired elements
    text = get_text(trimmed_tree, site_config["text_elements"])            
    return " ".join(text)

In [14]:
# if __name__=="__main__":
#     # Load up test pages
#     from util.load_test_data import get_test_pages
    
#     request_list = get_test_pages()

#     for page in request_list:
#         text = page_processer(page)
#         print(text)
#         print("".join(['--'for i in range(20)]))